# 8. DART 공시, 증권사 리포트 및 경제 데이터 수집하기

이번 장에서는 주식과 재무제표와 같은 금융 데이터 외에 투자에 도움이 될 수 있는 기업의 공시내용, 증권사 애널리스트의 분석 리포트 및 경제 데이터를 수집하는 방법에 대해 알아보도록 하겠습니다. 이는 나중에 배울 텔래그램과의 연동을 통해 실시간으로 내 텔레그램에 전송할 수도 있습니다.

## 8.1 DART의 Open API를 이용한 데이터 수집하기

DART(Data Analysis, Retrieval and Transfer System)는 금융감독원 전자공시시스템으로써, 상장법인 등이 공시서류를 인터넷으로 제출하고, 투자자 등 이용자는 제출 즉시 인터넷을 통해 조회할 수 있도록 하는 종합적 기업공시 시스템입니다. 홈페이지에서도 각종 공시내역을 확인할 수 있지만, 해당 사이트에서 제공하는 API를 이용할 경우 더욱 쉽게 공시 내용을 수집할 수 있습니다.

### 8.1.1 API Key발급 및 추가하기

먼저 https://opendart.fss.or.kr/에 접속한 후 [인증키 신청/관리] → [인증키 신청]을 통해 API Key를 발급 받습니다.

```{figure} image/06_dart_api_key.png
---
name: 06_dart_api_key
---
OpenAPI 인증키 신청
```

계정을 생성하고 이메일을 통해 이용자 등록을 한 후 로그인을 합니다. 그 후 [오픈API 이용현황]을 살펴보면 **API Key** 부분에 발급받은 Key가 있으며, 금일 몇번의 API를 요청했는지가 일일이용현황에 나옵니다. 하루 총 10,000번까지 데이터를 요청할 수 있습니다.

```{figure} image/dart_api_status.png
---
name: dart_api_status
---
OpenAPI 이용현황
```

다음으로 발급받은 API Key를 keyring 패키지를 이용해 저장합니다.

In [21]:
import keyring
keyring.set_password('Henry', 'dart_api_key', 'Your API Key')

### 8.1.2 고유번호 다운로드

Open API에서 각 기업의 데이터를 받기 위해서는 종목에 해당하는 고유번호를 알아야 합니다. 이에 대한 개발가이드는 아래 페이지에 나와 있습니다.

```
https://opendart.fss.or.kr/guide/detail.do?apiGrpCd=DS001&apiId=2019018
```

위 페이지의 내용을 코드로 나타내보도록 합니다.

In [13]:
import keyring
import requests as rq
from io import BytesIO
import zipfile

dart_api = keyring.get_password('Henry', 'dart_api_key')
codezip_url = 'https://opendart.fss.or.kr/api/corpCode.xml?crtfc_key='+dart_api
codezip_data = rq.get(codezip_url)

codezip_data.headers

{'Cache-Control': 'no-cache, no-store', 'Connection': 'keep-alive', 'Set-Cookie': 'WMONID=zEHEG-i2Toj; Expires=Tue, 15-Feb-2022 21:55:53 GMT; Path=/', 'Pragma': 'no-cache', 'Expires': '0', 'Content-Transfer-Encoding': 'binary', 'Content-Disposition': ': attachment; filename=CORPCODE.zip', 'Date': 'Mon, 15 Feb 2021 12:55:53 GMT', 'Content-Type': 'application/x-msdownload;charset=UTF-8', 'Content-Length': '1406062'}

1. https://opendart.fss.or.kr/api/corpCode.xml?crtfc_key= 뒤에 본인의 API 키를 입력합니다.
2. `get()` 함수를 통해 해당 페이지 내용을 받습니다.
3. 헤더를 확인해보면 attachment, 즉 파일이 첨부되어 있습니다. 이에 대해 좀더 자세히 알아보도록 하겠습니다.

In [14]:
codezip_data.headers['Content-Disposition']

': attachment; filename=CORPCODE.zip'

headers의 'content-disposition' 부분을 확인해보면 **CORPCODE.zip** 파일이 첨부되어 있습니다. 해당 파일의 압축을 풀어 첨부된 내용을 확인합니다.

In [15]:
codezip_file = zipfile.ZipFile(BytesIO(codezip_data.content))
codezip_file.namelist()

['CORPCODE.xml']

`BytesIO()` 함수를 통해 바이너리스트림 형태로 만든 후, `ZipFile()` 함수를 통해 압축을 풀어줍니다. **CORPCODE.xml**라는 파일이 존재하며, 이를 불러오도록 합니다.

In [16]:
code_data  = codezip_file.read('CORPCODE.xml').decode('utf-8')

해당 데이터는 XML 형태로 이루어져 있으므로, 데이터프레임 형태로 변경하도록 하겠습니다.

In [17]:
import xmltodict
import json
import pandas as pd

data_odict = xmltodict.parse(code_data)
data_dict = json.loads(json.dumps(data_odict))
data = data_dict.get('result').get('list')
corp_list = pd.DataFrame(data)

corp_list.head()

,corp_code,corp_name,stock_code,modify_date
0,00434003,다코,None,20170630
1,00434456,일산약품,None,20170630
2,00430964,굿앤엘에스,None,20170630
3,00432403,한라판지,None,20170630
4,00388953,크레디피아제이십오차유동화전문회사,None,20170630


1. xmltodict 패키지의 `parse()` 함수를 이용해 딕셔너리 형태로 변경합니다.
2. 위 데이터를 `dumps()` 함수를 통해 JSON 형태로 바꿔준 후, `loads()` 함수를 통해 불러옵니다.
3. `get()` 함수를 통해 result 내에서 list 부분만 불러옵니다.
4. 데이터프레임 형태로 변경해줍니다.

In [18]:
len(corp_list)

83571

종목수를 확인해보면 거래소의 상장 종목수보다 훨씬 많으며, 이는 stock_code 열이 빈 종목은 거래소에 상장되지 않은 종목이기 때문입니다.. 따라서 해당 데이터는 삭제하여 거래소 상장 종목만을 남긴 후, csv 파일로 저장하도록 합니다.

In [19]:
corp_list = corp_list[~corp_list.stock_code.isin([None])].reset_index(drop=True)
corp_list.to_csv('data/corp_list.csv')

### 8.1.3 공시검색

#### 8.1.3.1 전체 공시 검색 공시 검색

먼저 공시검색 API에 대한 이해를 위해 전체 종목의 공시를 수집하도록 하며, 해당 개발가이드는 아래 페이지에 나와 있습니다.

```
https://opendart.fss.or.kr/guide/detail.do?apiGrpCd=DS001&apiId=2019001
```

각종 요청인자를 통해 url을 생성 후 전송하여, 요청에 맞는 데이터를 받을 수 있습니다. 공시 검색에 해당하는 인자는 다음과 같습니다.


```{figure} image/dart_api_input.png
---
name: dart_api_input
---
OpenAPI 요청 인자 예시
```

```{figure} image/dart_api_exam.png
---
name: dart_api_exam
---
OpenAPI 테스트 예시
```

먼저 시작일과 종료일을 토대로 최근 공시 100건에 해당하는 url을 생성하도록 하겠습니다.

In [20]:
from datetime import date
from dateutil.relativedelta import relativedelta

bgn_date = (date.today() + relativedelta(days=-7)).strftime("%Y%m%d")
end_date = (date.today()).strftime("%Y%m%d")

notice_url = 'https://opendart.fss.or.kr/api/list.json?crtfc_key='+dart_api+'&bgn_de='+bgn_date+'&end_de='+end_date+'&page_no=1&page_count=100'

notice_data  = rq.get(notice_url)
notice_data_df = notice_data.json().get('list')
notice_data_df = pd.DataFrame(notice_data_df)

notice_data_df.tail()

,corp_code,corp_name,stock_code,corp_cls,report_nm,rcept_no,flr_nm,rcept_dt,rm
95,00118008,동원금속,018500,Y,분기보고서 (2020.12),20210215001359,동원금속,20210215,
96,00555874,제주항공,089590,Y,매출액또는손익구조30%(대규모법인은15%)이상변경,20210215801096,제주항공,20210215,유
97,00104856,삼성증권,016360,Y,증권신고서(채무증권),20210215001356,삼성증권,20210215,
98,01011395,레몬,294140,K,주주총회소집결의,20210215901040,레몬,20210215,코
99,01528965,디디아이백암로지스틱스위탁관리부동산투자회사,,E,비유동자산취득결정,20210215001355,디디아이백암로지스틱스위탁관리부동산투자회사,20210215,공


1. bgn_date에는 현재로부터 일주일 전을, end_date는 오늘 날짜를, 페이지별 건수에 해당하는 page_count에는 100을 입력하도록 합니다.
2. 그 후 홈페이지에 나와있는 예시에 맞게 url을 작성해주도록 합니다.
3. `get()` 함수를 통해 해당 페이지 내용을 받습니다.
4. `json()` 함수를 통해 JSON 데이터를 불러온 후, list 부분만을 선택합니다.
5. 데이터프레임 형태로 변경합니다.

데이터를 확인해보면 우리가 원하는 공시정보, 즉 일주일 전부터 오늘까지 100건의 공시 정보가 다운로드 되어 있습니다.

#### 8.1.3.2 특정 기업의 공시 검색

이번에는 고유번호를 추가하여 원하는 기업의 공시만 확인해보록 하겠습니다. 고유번호는 위에서 다운받은 **corp_list.csv** 파일을 통해 확인해볼 수 있으며, 예시로 살펴볼 삼성전자의 고유번호는 [00126380] 입니다.

In [21]:
corp_list[corp_list['corp_name'] == '삼성전자']

,corp_code,corp_name,stock_code,modify_date
3084,00126380,삼성전자,005930,20201209


In [22]:
bgn_date = (date.today() + relativedelta(days=-30)).strftime("%Y%m%d")
end_date = (date.today()).strftime("%Y%m%d")
corp_code  = '00126380'

notice_url_ss =  'https://opendart.fss.or.kr/api/list.json?crtfc_key='+dart_api+'&corp_code='+corp_code+'&bgn_de='+bgn_date+'&end_de='+end_date+'&page_no=1&page_count=100'

notice_data_ss  = rq.get(notice_url_ss)
notice_data_ss_df = notice_data_ss.json().get('list')
notice_data_ss_df = pd.DataFrame(notice_data_ss_df)

notice_data_ss_df.tail()

,corp_code,corp_name,stock_code,corp_cls,report_nm,rcept_no,flr_nm,rcept_dt,rm
11,00126380,삼성전자,005930,Y,수시공시의무관련사항(공정공시),20210128800075,삼성전자,20210128,유
12,00126380,삼성전자,005930,Y,현금ㆍ현물배당결정,20210128800069,삼성전자,20210128,유
13,00126380,삼성전자,005930,Y,연결재무제표기준영업(잠정)실적(공정공시),20210128800062,삼성전자,20210128,유
14,00126380,삼성전자,005930,Y,[기재정정]연결재무제표기준영업(잠정)실적(공정공시),20210128800050,삼성전자,20210128,유
15,00126380,삼성전자,005930,Y,횡령ㆍ배임사실확인,20210120800650,삼성전자,20210120,유


1. 시작일을 과거 30일로 수정하였으며, 기존 url에 &corp_code= 부분을 추가하였습니다.
2. 그 이후 진행과정인 이전과 동일합니다.

데이터 중 rcept_no는 공시번호에 해당하며, 해당 데이터를 이용해 공시에 해당하는 url에 접속을 할 수도 있습니다.

In [23]:
notice_url_exam  = notice_data_ss_df.loc[0, 'rcept_no']
notice_dart_url = 'http://dart.fss.or.kr/dsaf001/main.do?rcpNo='+notice_url_exam

print(notice_dart_url)

http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20210215001257


dart 홈페이지의 공시에 해당하는 url과 첫번째 공시에 해당하는 공시번호를 합쳐주도록 합니다. 위 url에 접속하여 해당 공시를 좀 더 자세하게 확인할 수 있습니다.

```{figure} image/dart_api_web.png
---
name: dart_api_web
---
공시 정보의 확인
```

### 8.1.4 사업보고서 주요 정보

API를 이용하여 사업보고서의 주요 정보 역시 다운로드 받을 수 있으며, 제공하는 목록은 다음과 같습니다.

```
https://opendart.fss.or.kr/guide/main.do?apiGrpCd=DS002
```
이 중 예시로써 [배당에 관한 사항]을 다운로드 받도록 하며, 개발가이드 페이지는 다음과 같습니다.

```
https://opendart.fss.or.kr/guide/detail.do?apiGrpCd=DS002&apiId=2019005
```
url 생성에 필요한 요청 인자는 다음과 같습니다.

```{table} 배당에 관한 사항 주요 인자
:name: div_input
| 키	| 명칭 | 설명 |
| --- | --- | --- |
| crtfc_key	| API 인증키 |	발급받은 인증키
| corp_code	| 고유번호 | 공시대상회사의 고유번호(8자리) |
| bsns_year	| 사업년도 | 사업연도(4자리) |
| reprt_code | 보고서 코드 | <ul>1분기보고서 : 11013</ul> <ul>반기보고서 : 11012 </ul> <ul>3분기보고서 : 11014</ul> <ul>사업보고서 : 11011 </ul> |
```

이를 바탕으로 삼성전자의 2019년 사업보고서를 통해 배당에 관한 사항을 살펴보도록 하겠습니다.

In [24]:
corp_code = '00126380'
bsns_year = '2019'
reprt_code = '11011'

url_div = 'https://opendart.fss.or.kr/api/alotMatter.json?crtfc_key='+dart_api+'&corp_code='+corp_code+'&bsns_year='+bsns_year+'&reprt_code='+reprt_code

div_data_ss = rq.get(url_div)
div_data_ss_df = div_data_ss.json().get('list')
div_data_ss_df = pd.DataFrame(div_data_ss_df)

div_data_ss_df.head()

,rcept_no,corp_cls,corp_code,corp_name,se,thstrm,frmtrm,lwfr,stock_knd
0,20200330003851,Y,00126380,삼성전자,주당액면가액(원),100,100,100,NaN
1,20200330003851,Y,00126380,삼성전자,(연결)당기순이익(백만원),"21,505,054","43,890,877","41,344,569",NaN
2,20200330003851,Y,00126380,삼성전자,(별도)당기순이익(백만원),"15,353,323","32,815,127","28,800,837",NaN
3,20200330003851,Y,00126380,삼성전자,(연결)주당순이익(원),"3,166","6,461","5,997",NaN
4,20200330003851,Y,00126380,삼성전자,현금배당금총액(백만원),"9,619,243","9,619,243","5,826,302",NaN


API 인증키, 고유번호, 사업년도, 보고서 코드에 각각 해당하는 데이터를 입력하여 url 생성하고, 앞에서 했던것과 동일한 방식으로 데이터를 불러옵니다. 데이터를 확인해보면, 사업보고서 중 배당에 관한 사항만이 나타나 있습니다. 위 url의 **alotMatter** 부분을 각 사업보고서에 해당하는 값으로 변경해주면 다른 정보 역시 동일한 방법으로 수집이 가능합니다.